In [1]:
import torch
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import tqdm
import numpy as np
from loguru import logger
import json

In [2]:
from torch import nn
import torch.nn.functional as F

In [3]:
logger.remove()
logger.add("log.txt")

1

In [4]:
DEVICE = "cuda:1"

## Load models for vector generation

## Load data

### Read vectors

In [81]:
embeddings = np.load("data/vectors.npy")

In [94]:
indices = np.arange(embeddings.shape[0]//3)

train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=42)

train_data = [embeddings[train_idx*3], embeddings[train_idx*3+1], embeddings[train_idx*3+2]]
test_data = [embeddings[test_idx*3], embeddings[test_idx*3+1], embeddings[test_idx*3+2]]

print(train_data[0].shape)
print(test_data[0].shape)

(69982, 768)
(29993, 768)


## Measure cosine similarity differences between pos and neg

In [117]:
def calculate_metrics(method, data, name):
    orig_shape = data[0].shape[1]
    pos_similarity = []
    neg_similarity = []

    queue = []
    anchors = method(data[0])
    positives = method(data[1])
    negatives = method(data[2])
   
    for i in tqdm.trange(len(anchors)):
        query, pos, neg = anchors[i], positives[i], negatives[i]
        pos_similarity.append(cosine_similarity([query], [pos])[0][0])
        neg_similarity.append(cosine_similarity([query], [neg])[0][0])

    pos_similarity = np.array(pos_similarity)
    neg_similarity = np.array(neg_similarity)
    
    ratio = np.sum(pos_similarity>neg_similarity)/len(pos_similarity)
    mean_diff = pos_similarity.mean()-neg_similarity.mean()
    mean_pos_sim = pos_similarity.mean()
    
    reduction_rate = orig_shape/len(anchors[0])
    return pd.DataFrame.from_dict([{
                        "ratio": ratio,
                        "mean_pos_sim": mean_pos_sim,
                        "mean_diff": mean_diff,
                        "method": name,
                        "reduction_rate": reduction_rate
                    }])

## Benchmarks

In [ ]:
metrics = []

### Baseline

In [120]:
metrics.append(calculate_metrics(lambda x: x, test_data, "CLS vector"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:11<00:00, 2722.30it/s]


In [ ]:
# lda_metrics = []
# embeddings = []

# train_data = data.sample(1000, random_state=42)

# for i in tqdm.trange(0, train_data.shape[0], 32):
#     embeddings.extend(embed(

In [40]:
dim = 64

# for idx in range(0, train.shape

In [42]:
for

,query,pos,neg
15773,what does dsm stand for,"In psychology and psychiatry, DSM stands for t...","military mk, what does mark stand for in milit..."
35290,what is social competence in early childhood,Social competence in early childhood is as a s...,The Early Years Learning Framework (EYLF) is d...
358404,stonefield construction phone number,A: The phone number for Stonefield Constructio...,The Building Official reviews applications for...
381806,what book did adam smith write,"Adam Smith FRSA was a Scottish economist, phil...",Was Adam Smith laissez-faire? What did it mean...
244560,how long can you take naltrexone for,You and your doctor have to decide this. Most ...,"Naloxone, also known as Narcan among other nam..."
...,...,...,...
178265,what happens in the first stage of seed germin...,Stage One - Hydration and Metabolism. In the f...,Germination is the process in which the seeds ...
168368,aberdeen airport code,Aberdeen International Airport (Scottish Gaeli...,Welcome to USZip.com. USZip.com USZip.com is a...
314866,how old till baby can sleep on stomach,You may find yourself wondering if your baby c...,Third Month Baby Milestones: Sleep. Your 3-mon...
472372,what are the sisters last names in in the time...,In the Time of the Butterflies Characters. BÃ©...,The nine brightest stars of the Pleiades are n...


### Averaging

In [115]:
def average_vec(dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = x.mean(axis=1).reshape(-1, 1)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [118]:
for rate in [4, 8, 16, 32, 64, 128]:
    method = average_vec(rate)
    metrics.append(calculate_metrics(method, test_data, f"Average parts ({rate})"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:10<00:00, 2991.96it/s]


In [121]:
pd.concat(metrics).sort_values("ratio").to_csv("results/benchmarks.csv")

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.743273,0.705321,0.226744,Average parts (128),128.0
0,0.822659,0.717699,0.233296,Average parts (64),64.0
0,0.884240,0.731807,0.232372,Average parts (32),32.0
0,0.922682,0.734765,0.231430,Average parts (16),16.0
0,0.944587,0.732207,0.229667,Average parts (8),8.0
0,0.959691,0.732543,0.229968,Average parts (4),4.0
0,0.968393,0.733018,0.228607,CLS vector,1.0


### Quantize (Shelved for later)

In [14]:
# def train_embeddings(model, optim):
#     batch_size = 32
    
#     def _inner(term):
#         total_loss = 0
#         embeddings = get_embeddings(term)
#         reshaped = embeddings.view(-1, model.input_dim)
#         _res = []
#         for i in range(0, len(reshaped), batch_size):
#             _batch = reshaped[i:i+batch_size]
#             res, loss = model(_batch)
            
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#             _res.append(res)
#             total_loss += loss.item()
            
#         res = torch.concat(_res)

#         return res.view(len(embeddings), -1), total_loss
#     return _inner

# def test_embeddings(model):
#     def _inner(term):
#         optim.zero_grad()

#         embeddings = get_embeddings(term)
#         reshaped = embeddings.view(-1, model.input_dim)
        
#         res = model.reduce(reshaped)
#         return res.view(len(embeddings), -1)
#     return _inner

In [41]:
# class VectorQuantize(nn.Module):
#     def __init__(self, input_dim, codebook_dim, codebook_size, gamma=1):
#         super().__init__()
#         codebook_ = torch.rand(codebook_size, codebook_dim)

#         self.input_dim = input_dim
#         self.codebook = torch.nn.Parameter(codebook_)
#         # self.project = nn.Linear(input_dim, codebook_dim)
#         self.project = nn.Sequential(nn.Linear(input_dim, 32),
#                                       nn.ReLU(),
#                                       nn.Linear(32, codebook_dim))
#         self.expected_dist = (torch.ones(len(self.codebook))/len(self.codebook)).view(1, -1)
#         self.gamma = gamma
#         self.hinge_margin = torch.tensor(0)
#         self.loss = []
        
#     def var_loss(self, x):
#         var = torch.sqrt(x.var(0)+1e-6)
#         loss = torch.maximum(self.hinge_margin, self.gamma-var).mean()
#         return loss
        
#     def forward(self, x):
#         normalized = F.normalize(x)
#         projected = self.project(normalized)
#         x = F.normalize(projected)
        
#         codes_ = F.normalize(self.codebook)
#         dist = x@codes_.T
#         # dist = F.softmax(dist, dim=1)
#         # codebook_similarity_loss = F.cross_entropy(dist.mean(0).view(1, -1), self.expected_dist)
#         codebook_similarity_loss = self.var_loss(dist)
        
#         dist = x@codes_.T.detach()
#         maximum = dist.argmax(axis=1)
#         embedding = F.embedding(maximum, self.codebook)
#         codebook_difference_loss = F.cosine_embedding_loss(projected, embedding, torch.ones(len(x)))
#         # codebook_difference_loss = F.cosine_embedding_loss(x, embedding, torch.ones(len(x)), 0.1)
        
#         variance_loss = self.var_loss(x)
        
#         self.loss = [codebook_difference_loss, codebook_similarity_loss, variance_loss]
        
#         # loss = codebook_difference_loss + codebook_similarity_loss + variance_loss
#         loss = variance_loss
        
#         return x, loss
    
#     def reduce(self, x):
#         normalized = F.normalize(x)
#         return self.project(normalized).cpu().detach()

In [70]:
# vec = torch.rand(1, 64)

# quantizer = VectorQuantize(64, 32, 4)
# quantizer.project.modules().__next__()[0].parameters().__next__()

In [71]:
# optim = torch.optim.Adam(quantizer.parameters(), lr=3e-2)
# _data = train_data.sample(1000)
# for i in range(5):
#     print(calculate_metrics(train_embeddings(quantizer, optim), _data, ""))
#     print(quantizer.loss)

In [72]:
# vec = torch.rand(3, 64)
# projected = quantizer.reduce(vec)
# vec, projected
# quantizer.project.modules().__next__()[0].parameters().__next__()

In [73]:
# vec = torch.rand(8, 16)
# expected_dist = (torch.ones(len(quantizer.codebook))/len(quantizer.codebook)).view(1, -1)

# normalized = F.normalize(vec)
# projected = quantizer.project(normalized)
# x = F.normalize(projected)

# codes_ = F.normalize(quantizer.codebook)
# dist = x@codes_.T
# quantizer.var_loss(dist)

In [123]:
# for dims in [(16, 2),
#              (32, 4),
#              (32, 2),
#              (64, 8),
#              (64, 4),
#              (64, 2),
#              (128, 16),
#              (128, 8),
#              (128, 4)]:
#     quantizer = VectorQuantize(dims[0], dims[1], 64)
#     optim = torch.optim.Adam(quantizer.parameters(), lr=3e-4)

#     calculate_metrics(train_embeddings(quantizer, optim), train_data.sample(10000), "")

#     metrics.append(calculate_metrics(test_embeddings(quantizer), test_data.sample(10000), f"Vector quantizer ({dims[0]}x{dims[1]}), normalized codebook, fixed"))
#     print(metrics[-1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:14<00:00, 134.10it/s]


      ratio       mse  mean_diff  diff_of_means  \
0  0.938452  0.004588   0.054719       0.054719   

                                              method  reduction_rate  
0  Vector quantizer (16x2), normalized codebook, ...             8.0  


 25%|████████████████████████████████████████▋                                                                                                                          | 2493/10000 [00:18<00:55, 135.73it/s]


KeyboardInterrupt: 

In [112]:
avg = pd.concat(metrics).groupby("method").mean()
avg["compression"] = avg.index.map(lambda x: float(x.split("x")[0])/float(x.split("x")[1].split()[0]))

In [113]:
avg.sort_values("ratio")

,ratio,mse,mean_diff,diff_of_means,compression
method,,,,,
32x4 (256) proper,0.93352,0.000079,0.006874,0.006874,8.0
